## 1. Rental Data

In [1]:
import pandas as pd
import os

# Set up file paths and directories
base_dir = '../..'
data_dir = os.path.join(base_dir, 'data')
raw_rent_dir = os.path.join(data_dir, 'raw', 'rent_data') 
os.makedirs(raw_rent_dir, exist_ok=True)
landing_dir = os.path.join(data_dir, 'landing')
raw_other_dir = os.path.join(data_dir, 'raw', 'other_data')
raw_dir = os.path.join(data_dir, 'raw')
curated_dir = os.path.join(data_dir, 'curated')

In [2]:

# Step 1: Load the downloaded rental data Excel file from raw folder
rental_data_path = os.path.join(landing_dir, 'other_data', 'Quarterly_median_rent_March_2024.xlsx')

# Step 2: Load the 'All properties' sheet from the Excel file, skipping the first two rows, and drop NaN columns
df_cleaned = pd.read_excel(rental_data_path, sheet_name='All Properties', header=2).dropna(axis=1, how='all')

# Step 3: Forward-fill the 'Region' and 'Suburb' columns to ensure no missing values
df_cleaned['Unnamed: 0'] = df_cleaned['Unnamed: 0'].ffill()  # Region
df_cleaned['Unnamed: 1'] = df_cleaned['Unnamed: 1'].ffill()  # Suburb

# Step 4: Manually read the dates from the second row (assuming the date is in the even columns starting from column 2)
date_headers = pd.read_excel(rental_data_path, sheet_name='All Properties', header=None).iloc[1, 2::2]

# Step 5: Create an empty DataFrame to store transformed data
transformed_data = pd.DataFrame()

# Step 6: Iterate through rows to combine dates with count and median data
for i, row in df_cleaned.iterrows():
    region = row['Unnamed: 0']  # Region
    suburb = row['Unnamed: 1']  # Suburb
    row_data = {'Region': region, 'Suburb': suburb}

    # Loop over the columns, skipping by 2 (Count and Median pairs)
    for j in range(2, len(row), 2):
        date = date_headers.iloc[(j - 2) // 2]  # Extract the date corresponding to this column
        count = row[j]  # Count value
        median = row[j + 1]  # Median value

        # Create column names for each date's count and median
        count_col = f'{date} count'
        median_col = f'{date} median'

        # Add the count and median values to the row_data dictionary
        row_data[count_col] = count
        row_data[median_col] = median

    # Append the row data to the transformed_data DataFrame
    transformed_data = transformed_data.append(row_data, ignore_index=True)

# Step 7: Clean and convert rent data
def clean_rent_data(df):
    for col in df.columns:
        if 'count' in col or 'median' in col:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert non-numeric values to NaN
    return df

# Step 8: Apply cleaning and preprocess the transformed rental data
cleaned_rent_data = clean_rent_data(transformed_data)

# Step 9: Drop rows where 'Region' or 'Suburb' is missing and ensure no NaN values in important columns
cleaned_rent_data = cleaned_rent_data.dropna(subset=['Region', 'Suburb'])

# Drop rows where any 'count' or 'median' column contains NaN values
count_columns = [col for col in cleaned_rent_data.columns if 'count' in col]
median_columns = [col for col in cleaned_rent_data.columns if 'median' in col]
cleaned_rent_data = cleaned_rent_data.dropna(subset=count_columns + median_columns)

# Step 10: Load the LGA to SA2 correspondence data from raw folder
lga_sa2_file_path = os.path.join(landing_dir, 'other_data', 'CG_SA2_2021_LGA_2022.csv')
correspondence_data = pd.read_csv(lga_sa2_file_path)[['SA2_NAME_2021', 'LGA_NAME_2022']]

# Step 11: Standardize the 'Suburb' and 'LGA_NAME_2022' columns for merging
# Remove spaces, hyphens, and convert to lowercase for consistent merging
cleaned_rent_data['Suburb_standardized'] = cleaned_rent_data['Suburb'].str.replace('-', '').str.replace(' ', '').str.lower()
correspondence_data['LGA_standardized'] = correspondence_data['LGA_NAME_2022'].str.replace('-', '').str.replace(' ', '').str.lower()

# Step 12: Merge the cleaned rent data with the SA2-LGA correspondence data
merged_rent_data = pd.merge(cleaned_rent_data, correspondence_data, left_on='Suburb_standardized', right_on='LGA_standardized', how='left')

# Step 13: Clean up and remove unnecessary columns after the merge
merged_rent_data = merged_rent_data.drop(columns=['Suburb_standardized', 'LGA_standardized', 'LGA_NAME_2022', 'Region'])

# Rename the 'SA2_NAME_2021' column to 'SA2' for consistency
merged_rent_data.rename(columns={'SA2_NAME_2021': 'SA2'}, inplace=True)

# Step 14: Reorganize the columns so that 'SA2' is in the second position
cols = list(merged_rent_data.columns)
cols.insert(1, cols.pop(cols.index('SA2')))  # Move 'SA2' to the second position
merged_rent_data = merged_rent_data[cols]

# Step 15: Save the final processed rent data to 'curated_01'
output_rent_path = os.path.join(raw_rent_dir, 'final_rent_data.csv')
merged_rent_data.to_csv(output_rent_path, index=False)

# Display the first few rows of the cleaned and merged rent data
print(merged_rent_data.head())
print(f"Final rent data saved to {output_rent_path}")


/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/4043624732.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append(row_data, ignore_index=True)
/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/4043624732.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append(row_data, ignore_index=True)
/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/4043624732.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append(row_data, ignore_index=True)
/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/4043624732.py:38: FutureWarning: The frame.append method is depre

        Suburb                  SA2  Jun 1999 count  Jun 1999 median  \
0  Colac-Otway                Colac            99.0            115.0   
1  Colac-Otway      Colac Surrounds            99.0            115.0   
2  Colac-Otway                Otway            99.0            115.0   
3  Corangamite           Camperdown            36.0             93.0   
4  Corangamite  Corangamite - North            36.0             93.0   

   Sep 1999 count  Sep 1999 median  Dec 1999 count  Dec 1999 median  \
0            80.0            113.0            82.0            115.0   
1            80.0            113.0            82.0            115.0   
2            80.0            113.0            82.0            115.0   
3            44.0             90.0            47.0            100.0   
4            44.0             90.0            47.0            100.0   

   Mar 2000 count  Mar 2000 median  ...  Mar 2023 count  Mar 2023 median  \
0            96.0            115.0  ...            86.0         

## 2. Criminal data

In [3]:
import pandas as pd
import os


# Step 1: Load the pre-downloaded criminal data from the raw folder
criminal_data_path = os.path.join(raw_other_dir, 'criminal_data.csv')
criminal_data = pd.read_csv(criminal_data_path)

# Step 2: Create a new column combining 'Year' and 'Year ending' for pivoting
criminal_data['Year_Month'] = criminal_data['Year'].astype(str) + ' ' + criminal_data['Year ending']

# Step 3: Pivot the criminal data by 'Postcode' and 'Year_Month' to create a table of counts
criminal_pivot = pd.pivot_table(criminal_data, 
                                index=['Postcode'], 
                                columns='Year_Month', 
                                aggfunc='size', 
                                fill_value=0).reset_index()

# Step 4: Calculate the median criminal count for each Postcode across all years
criminal_data_postcode_median = criminal_pivot.median(axis=1)
criminal_data_postcode = criminal_pivot[['Postcode']]
criminal_data_postcode['Median Criminal Count'] = criminal_data_postcode_median

# Step 5: Load the POSTCODE to SA2 mapping data from raw folder
postcode_sa2_file_path = os.path.join(landing_dir, 'other_data', 'CG_POSTCODE_2021_SA2_2021.xlsx')
postcode_sa2_data = pd.read_excel(postcode_sa2_file_path, usecols=['POSTCODE', 'SA2_NAME_2021'])

# Step 6: Merge the criminal data with the POSTCODE to SA2 correspondence file
merged_criminal_data = pd.merge(criminal_data_postcode, postcode_sa2_data, left_on='Postcode', right_on='POSTCODE', how='left')

# Step 7: Drop unnecessary columns after merging
merged_criminal_data = merged_criminal_data[['SA2_NAME_2021', 'Median Criminal Count']]

# Step 8: Aggregate the merged criminal data by SA2
aggregated_criminal_data = merged_criminal_data.groupby('SA2_NAME_2021').agg({
    'Median Criminal Count': 'mean'  # Aggregating the median count by SA2
}).reset_index()

# Step 9: Rename 'SA2_NAME_2021' to 'SA2' for consistency across datasets
aggregated_criminal_data.rename(columns={'SA2_NAME_2021': 'SA2'}, inplace=True)

# Step 10: Save the final aggregated criminal data to 'curated_01'
aggregated_criminal_data_file = os.path.join(raw_rent_dir, 'aggregated_criminal_data_sa2.csv')
aggregated_criminal_data.to_csv(aggregated_criminal_data_file, index=False)

# Display the first few rows of the aggregated criminal data
print(aggregated_criminal_data.head())
print(f"Aggregated criminal data saved to {aggregated_criminal_data_file}")


/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/3529672311.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  criminal_data_postcode['Median Criminal Count'] = criminal_data_postcode_median


             SA2  Median Criminal Count
0     Abbotsford              52.000000
1   Airport West              85.000000
2    Albert Park              85.500000
3  Albury - East             121.000000
4      Alexandra              22.555556
Aggregated criminal data saved to ../../data/raw/rent_data/aggregated_criminal_data_sa2.csv


## 3. Region Data by SA2

In [4]:
import pandas as pd
import os
import geopandas as gpd
from geopy.distance import geodesic


# Load the facility and region growth rates data from CSV files
facility_file_path = os.path.join(raw_dir, 'facility_count', 'facility_merged.csv')
region_growth_rates_file_path = os.path.join(raw_dir, 'region_data', 'region_growth_rates.csv')
range_region_data_file_path = os.path.join(raw_dir, 'region_data', 'range_region_data.csv')

facility_data = pd.read_csv(facility_file_path)
region_growth_rates_data = pd.read_csv(region_growth_rates_file_path)
range_region_data = pd.read_csv(range_region_data_file_path)

# Step 1: Standardize region names for consistency across datasets
facility_data['region_standardized'] = facility_data['region'].str.replace(' ', '')
region_growth_rates_data['region_standardized'] = region_growth_rates_data['Region'].str.replace(' ', '')
range_region_data['region_standardized'] = range_region_data['Region'].str.replace(' ', '')

# Step 2: Merge facility data and region growth rates using standardized region names
merged_data = pd.merge(facility_data, region_growth_rates_data, on='region_standardized', how='left')

# Merge the result with range region data on the standardized region names
merged_data = pd.merge(merged_data, range_region_data, on='region_standardized', how='left')

# Step 3: Drop rows where the population is NaN to ensure regions are only from Victoria
merged_data = merged_data.dropna(subset=['population'])

# Step 4: Drop unnecessary columns after the merge (e.g., 'region_standardized', 'Region_x', 'Region_y')
merged_data = merged_data.drop(columns=['region_standardized', 'Region_x', 'Region_y'])

# Step 5: Rename 'region' column to 'SA2' if necessary
if 'region' in merged_data.columns:
    merged_data.rename(columns={'region': 'SA2'}, inplace=True)

# Step 6: Load and process the SA2 shapefile to calculate the distance to Melbourne CBD
shapefile_path = os.path.join(landing_dir, 'region_data', 'sa2_dataset', 'sa2_unzip', 'SA2_2021_AUST_GDA2020.shp')
sa2_data = gpd.read_file(shapefile_path)

# Define Melbourne CBD coordinates
melbourne_cbd_coords = (-37.8124, 144.9623)

# Calculate centroids for each SA2 region
sa2_data['centroid'] = sa2_data.geometry.centroid
sa2_data['centroid_lat'] = sa2_data['centroid'].y
sa2_data['centroid_lon'] = sa2_data['centroid'].x

# Filter out rows with missing centroid values
sa2_data_valid = sa2_data.dropna(subset=['centroid_lat', 'centroid_lon'])

# Function to calculate the distance to Melbourne CBD
def calculate_distance_to_cbd(row):
    centroid_coords = (row['centroid_lat'], row['centroid_lon'])
    return geodesic(centroid_coords, melbourne_cbd_coords).km

# Step 7: Apply the distance calculation to each SA2 region
sa2_data_valid['distance_to_cbd'] = sa2_data_valid.apply(calculate_distance_to_cbd, axis=1)

# Step 8: Merge the SA2 data with the aggregated region data
merged_sa2_data = merged_data.merge(sa2_data_valid[['SA2_NAME21', 'distance_to_cbd']], left_on='SA2', right_on='SA2_NAME21')

# Step 9: Drop any unnecessary columns after the merge (e.g., 'SA2_NAME21')
merged_sa2_data = merged_sa2_data.drop(columns=['SA2_NAME21'])

# Step 10: Save the final processed region data with distance to Melbourne CBD
final_output_path = os.path.join(raw_rent_dir, 'final_region_data_with_distance.csv')
merged_sa2_data.to_csv(final_output_path, index=False)

# Display the first few rows to verify the final result
print(merged_sa2_data.head())
print(f"Final region data saved to {final_output_path}")


/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/2323979638.py:45: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sa2_data['centroid'] = sa2_data.geometry.centroid


                    SA2  entertainments_count  hospital_count  park_count  \
0  Melbourne CBD - East                 317.0             NaN         2.0   
1  Melbourne CBD - West                 153.0             NaN         3.0   
2               Fitzroy                 135.0             2.0        13.0   
3               Carlton                  97.0             1.0        21.0   
4               Geelong                  94.0             3.0        27.0   

   psf_count  school2_count  school1_count  shop_count  stop_count  \
0        3.0            7.0            2.0        10.0        32.0   
1        3.0            4.0            2.0         5.0        67.0   
2        1.0            6.0            3.0        14.0        35.0   
3        1.0           10.0            2.0         9.0        52.0   
4        2.0           11.0            7.0         4.0        94.0   

   population  ...  nUnEmployed_2011  nUnEmployed_2016  nUnEmployed_2021  \
0 -209.314286  ...             400.0    

/Users/xxu/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## 4. Merge All

In [5]:


# Step 1: Load the preprocessed criminal data (aggregated by SA2)
aggregated_criminal_data_file = os.path.join(raw_dir, 'rent_data','aggregated_criminal_data_sa2.csv')
criminal_data = pd.read_csv(aggregated_criminal_data_file)

# Step 2: Load the preprocessed rent data (merged with SA2)
rent_data_file = os.path.join(raw_dir, 'rent_data', 'final_rent_data.csv')
rent_data = pd.read_csv(rent_data_file)

# Step 3: Load the region data (aggregated with distance to Melbourne CBD)
region_data_file = os.path.join(raw_dir, 'rent_data', 'final_region_data_with_distance.csv')
region_data = pd.read_csv(region_data_file)

# Display the first few rows to verify the data
print("Criminal Data: ")
print(criminal_data.head())
print("Rent Data: ")
print(rent_data.head())
print("Region Data: ")
print(region_data.head())

# Step 4: Merge rent data with preprocessed criminal data on 'SA2'
merged_data = pd.merge(rent_data, criminal_data, on='SA2', how='left')

# Step 5: Merge the result with region data on 'SA2'
final_merged_data = pd.merge(merged_data, region_data, on='SA2', how='left')

# Display the first few rows of the final merged dataset to verify the merge
print("Final Merged Data: ")
print(final_merged_data.head())

# Step 6: Clean the merged dataset
# Drop any rows where the 'SA2' column is blank
final_merged_data = final_merged_data.dropna(subset=['SA2'])

# Fill missing values within each 'Suburb' (or 'LGA') using the median for each group
final_merged_data = final_merged_data.groupby('Suburb').apply(lambda group: group.fillna(group.median()))

# Rename the 'Suburb' column to 'LGA' for consistency
final_merged_data.rename(columns={'Suburb': 'LGA'}, inplace=True)

# Step 7: Check if any missing values are left in the dataset
missing_values = final_merged_data.isnull().sum().sum()

if missing_values == 0:
    print("No missing values left in the data.")
else:
    print(f"There are still {missing_values} missing values left in the data.")

# Step 8: Drop duplicates based on 'SA2' to ensure unique records per region
final_merged_data = final_merged_data.drop_duplicates(subset='SA2', keep='first')

# Remove rows where 'SA2' contains the word 'industrial' (case insensitive)
final_merged_data = final_merged_data[~final_merged_data['SA2'].str.contains('industrial', case=False, na=False)]



output_final_path = os.path.join(curated_dir, 'final_merged_data_sa2.csv')
final_merged_data.to_csv(output_final_path, index=False)

print(f"Final merged data saved to {output_final_path}")


Criminal Data: 
             SA2  Median Criminal Count
0     Abbotsford              52.000000
1   Airport West              85.000000
2    Albert Park              85.500000
3  Albury - East             121.000000
4      Alexandra              22.555556
Rent Data: 
        Suburb                  SA2  Jun 1999 count  Jun 1999 median  \
0  Colac-Otway                Colac            99.0            115.0   
1  Colac-Otway      Colac Surrounds            99.0            115.0   
2  Colac-Otway                Otway            99.0            115.0   
3  Corangamite           Camperdown            36.0             93.0   
4  Corangamite  Corangamite - North            36.0             93.0   

   Sep 1999 count  Sep 1999 median  Dec 1999 count  Dec 1999 median  \
0            80.0            113.0            82.0            115.0   
1            80.0            113.0            82.0            115.0   
2            80.0            113.0            82.0            115.0   
3            44

/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/449432.py:36: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  final_merged_data = final_merged_data.groupby('Suburb').apply(lambda group: group.fillna(group.median()))
/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/449432.py:36: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  final_merged_data = final_merged_data.groupby('Suburb').apply(lambda group: group.fillna(group.median()))
/var/folders/8_/gylgnm051zv9rk4m5lzq6c880000gn/T/ipykernel_30975/44943

There are still 29 missing values left in the data.
Final merged data saved to ../../data/curated/final_merged_data_sa2.csv
